In [300]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns

Upload the labels.csv and processed_counts.csv files to colab or your local workspace.

This data associates a cell barcode, such as "AAAGCCTGGCTAAC-1", to a certain cell type label, such as "CD14+ Monocyte". For each cell barcode, there are also log RNA seq counts of 765 different genes, such as HES4.

label.csv stores the association between a cell barcode and a cell type label.

processed_counts.csv stores the normalized log read counts for each cell, where each row represents a single cell, and each column represents a gene.

In [301]:
labels_pd = pd.read_csv("labels.csv")
counts_pd = pd.read_csv("processed_counts.csv")

In [302]:
labels_pd.head()

,index,bulk_labels
0,AAAGCCTGGCTAAC-1,CD14+ Monocyte
1,AAATTCGATGCACA-1,Dendritic
2,AACACGTGGTCTTT-1,CD56+ NK
3,AAGTGCACGTGCTA-1,CD4+/CD25 T Reg
4,ACACGAACGGAGTG-1,Dendritic


In [303]:
labels_pd.shape

(700, 2)

In [304]:
counts_pd.head()

,Unnamed: 0,HES4,TNFRSF4,SSU72,PARK7,RBP7,SRM,MAD2L2,AGTRAP,TNFRSF1B,...,ATP5O,MRPS6,TTC3,U2AF1,CSTB,SUMO3,ITGB2,S100B,PRMT2,MT-ND3
0,AAAGCCTGGCTAAC-1,-0.326,-0.191,-0.728,-0.301,3.386,-0.531,2.016,3.377,4.841,...,-0.146,-0.532,-0.341,0.303,1.404,4.294,0.519,-0.21,-0.636,4.011
1,AAATTCGATGCACA-1,1.171,-0.191,0.795,-1.200,-0.174,-0.531,1.889,-0.486,-0.459,...,-1.136,-0.532,-0.341,-0.905,2.849,-0.585,1.172,-0.21,2.630,-0.490
2,AACACGTGGTCTTT-1,-0.326,-0.191,0.483,-1.200,-0.174,-0.531,-0.451,0.971,-0.459,...,-1.136,2.606,-0.341,-0.905,-0.455,-0.585,0.722,-0.21,0.663,-0.490
3,AAGTGCACGTGCTA-1,-0.326,-0.191,1.134,-0.157,-0.174,-0.531,-0.451,-0.486,-0.459,...,1.161,-0.532,-0.341,-0.905,-0.119,-0.585,0.766,-0.21,-0.636,-0.490
4,ACACGAACGGAGTG-1,-0.326,-0.191,-0.728,-0.607,-0.174,-0.531,-0.451,0.787,-0.459,...,-1.136,0.839,1.679,-0.108,-0.534,-0.585,-0.007,-0.21,-0.636,-0.490


In [305]:
counts_pd.shape

(700, 766)

Shuffle your data. Make sure your labels and the counts are shuffled together.

Split into train and test sets (80:20 split)

In [306]:
# create random array for shuffling
index_a = np.arange(len(labels_pd))
np.random.shuffle(index_a)

# shuffle rows of both datasets based on array
labels_pd = labels_pd.iloc[index_a]
counts_pd = counts_pd.iloc[index_a]

# print
print(labels_pd)
print(counts_pd)

                index                   bulk_labels
232  GGGCAAGAAGGCGA-3  CD8+/CD45RA+ Naive Cytotoxic
349  TTTCAGTGTCACGA-4                CD14+ Monocyte
74   TAACATGACTTGAG-1                     Dendritic
29   CATCCCGATCTGGA-1                     Dendritic
426  TCGGCACTGTTGAC-5                     Dendritic
..                ...                           ...
282  AGACCTGAGACGTT-4                     Dendritic
445  AGACCTGATACTTC-6                CD14+ Monocyte
454  ATTGAAACAGATCC-6                CD14+ Monocyte
510  TGGTTACTTGCATG-6                      CD56+ NK
413  GGCTAAACTCTTTG-5           CD4+/CD45RO+ Memory

[700 rows x 2 columns]
           Unnamed: 0   HES4  TNFRSF4  SSU72  PARK7   RBP7    SRM  MAD2L2   
232  GGGCAAGAAGGCGA-3 -0.326   -0.191  0.599  0.286 -0.174 -0.531   1.588  \
349  TTTCAGTGTCACGA-4  1.013   -0.191 -0.728  0.325 -0.174 -0.531  -0.451   
74   TAACATGACTTGAG-1 -0.326   -0.191  0.238 -0.659 -0.174  3.125  -0.451   
29   CATCCCGATCTGGA-1  1.076   -0.191  0.698

In [307]:
labels_train, labels_test, counts_train, counts_test = train_test_split(labels_pd, counts_pd, test_size=0.2, random_state=42)

In [308]:
print("Train set shapes: ", labels_train.shape, counts_train.shape)
print("Test set shapes: ", labels_test.shape, counts_test.shape)

Train set shapes:  (560, 2) (560, 766)
Test set shapes:  (140, 2) (140, 766)


Create a fully connected neural network for your autoencoder. Your latent space can be of any size less than or equal to 64. Too large may result in a poor visualization, and too small may result in high loss. 32 is a good starting point.

Consider using more than 1 hidden layer, and a sparcity constraint (l1 regularization).

Have an encoder model which is a model of only the layers for the encoding.

In [309]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, latent_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, latent_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_size),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [310]:
# load training and testing datasets
labels_train, labels_test, counts_train, counts_test = train_test_split(labels_pd, counts_pd, test_size=0.2, random_state=42)

# only select columns with numerical data
counts_train = counts_train.iloc[:, 1:].values
counts_test = counts_test.iloc[:, 1:].values

# convert data to Torch tensor
counts_train = torch.tensor(counts_train, dtype=torch.float32)
counts_test = torch.tensor(counts_test, dtype=torch.float32)

# define autoencoder loss function
input_size = counts_train.shape[1]
latent_size = 32
model = Autoencoder(input_size, latent_size)
criterion = nn.MSELoss()

# use L1 regularization for optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

Train your autoencoding using MSE loss.

Finally, identify the parameters which don't overfit, and use the same model architecture and train on all of the data together.

With a latent space size of 32, aim for 0.9 MSE loss on your test set, 0.95 with regularization. You will not be graded strictly on a loss cutoff.

In [ ]:
# run autoencoder on train set
n_epochs = 100
batch_size = 32
for epoch in range(n_epochs):
    train_loss = 0
    for i in range(0, counts_train.shape[0], batch_size):
        inputs = counts_train[i:i+batch_size]
        model.zero_grad()
        encoded, decoded = model(inputs)
        loss = criterion(decoded, inputs) + 1e-3 * torch.mean(torch.abs(encoded))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.shape[0]
    train_loss /= counts_train.shape[0]
    print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, n_epochs, train_loss))

Epoch [1/100], Train Loss: 1.0000
Epoch [2/100], Train Loss: 0.9914
Epoch [3/100], Train Loss: 0.9787
Epoch [4/100], Train Loss: 0.9708
Epoch [5/100], Train Loss: 0.9612
Epoch [6/100], Train Loss: 0.9543
Epoch [7/100], Train Loss: 0.9500
Epoch [8/100], Train Loss: 0.9463
Epoch [9/100], Train Loss: 0.9424
Epoch [10/100], Train Loss: 0.9384
Epoch [11/100], Train Loss: 0.9368
Epoch [12/100], Train Loss: 0.9358
Epoch [13/100], Train Loss: 0.9353
Epoch [14/100], Train Loss: 0.9351
Epoch [15/100], Train Loss: 0.9329
Epoch [16/100], Train Loss: 0.9314
Epoch [17/100], Train Loss: 0.9299
Epoch [18/100], Train Loss: 0.9294
Epoch [19/100], Train Loss: 0.9303
Epoch [20/100], Train Loss: 0.9287
Epoch [21/100], Train Loss: 0.9272
Epoch [22/100], Train Loss: 0.9260
Epoch [23/100], Train Loss: 0.9255
Epoch [24/100], Train Loss: 0.9238
Epoch [25/100], Train Loss: 0.9227
Epoch [26/100], Train Loss: 0.9210
Epoch [27/100], Train Loss: 0.9207
Epoch [28/100], Train Loss: 0.9213
Epoch [29/100], Train Loss: 0

In [ ]:
# run autoencoder on test set
test_loss = 0
with torch.no_grad():
    for i in range(0, counts_test.shape[0], batch_size):
        inputs = counts_test[i:i+batch_size]
        encoded, decoded = model(inputs)
        loss = criterion(decoded, inputs)
        test_loss += loss.item() * inputs.shape[0]
test_loss /= counts_test.shape[0]
print('Test Loss: {:.4f}'.format(test_loss))

Use PCA and t-SNE on the dataset.

Then use PCA on the latent space representation of the dataset.

Plot all of these.

In [ ]:
# Load the data
counts_pd1 = pd.read_csv("processed_counts.csv", index_col=0)

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(counts_pd1)

In [ ]:
# t-SNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
X_tsne = tsne.fit_transform(X_pca)

Compare the results of PCA, t-SNE, and your autoencoder as ways to visualize the data.

In [ ]:
# Define the y variable for coloring the plot
y = labels_pd.iloc[:, 1].values

# plot the PCA results
sns.scatterplot(
    x=X_pca[:,0], y=X_pca[:,1],
    hue=y,
    palette=sns.color_palette("hls", 10),
    legend=False,
    alpha=0.75
)
plt.show()

"""
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA Plot')
plt.show()
"""

In [ ]:
counts_train.shape

In [ ]:
# Extract cell types from labels_train
cell_types = labels_train.iloc[:, 1]

# Get the encoded representation of the dataset
with torch.no_grad():
    encoded, _ = model(counts_train)
    encoded = encoded.numpy()

# Perform PCA on the encoded data
pca = PCA(n_components=2)
X_pca = pca.fit_transform(encoded)

# Plot the PCA-reduced data with color-coded cell types
fig, ax = plt.subplots()
for ct in cell_types.unique():
    ix = cell_types == ct
    ax.scatter(X_pca[ix, 0], X_pca[ix, 1], label=ct)

plt.show()

In [ ]:
# plot the t-SNE results
sns.scatterplot(
    x=X_tsne[:,0], y=X_tsne[:,1],
    hue=y,
    palette=sns.color_palette('hls', 10),
    legend=False,
    alpha=0.75
)
plt.show()